Notebook developed by Zeljko Ivezic using
original python implementation by Haley Stewart:

 
### Aluminum Sphere Extreme Temp Case
####     Aluminum sphere in LEO, hot and cold equilibrium cases.
####   Ability to run equilibrium calc multiple times with different coatings.
####    Plots Dark Side temp over time using Euler method
#### 11/20/20, Haley Stewart
 

## Improvements: 
- odvojiti fizikalne konstante i drugi nepromjenljivi input (e.g. solarna konstanta) od 
 stvarnog inputa (npr. Q_int)
- spremiti tekstualni output u tekst file, sa datumom i kopijom inputa (ili imena 
 input file-a, sto bi bila bolja opcija) tako da znamo sto smo racunali
- glavne slike spremiti kao png, sa imenom filea koji sadrzava neku input
 varijablu, npr. TempVStime_simulation1.png



     
     

In [422]:
import os 
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from astropy.table import Table
from astroML.filters import savitzky_golay, wiener_filter
%matplotlib inline

In [423]:
def fluxToHeat(flux, absorptivity, Area_circle):
    return flux*absorptivity*Area_circle

def eqTemp( Q_in, Area_sphere, emissivity, sigma):
    Temp = (Q_in/(sigma*emissivity*Area_sphere))**(1/4)
    #print(case +' Eq. Temp: ' + str('%.2f'%(Temp)) + 'K = ' + str('%.2f'%(Temp-KelToCel)) + u'\u2103')
    return Temp

def printEqTemp(case, Temp, KelToCel):
    print(case +' Eq. Temp: ' + str('%.2f'%(Temp)) + 'K = ' + str('%.2f'%(Temp-KelToCel)) + u'\u2103')

def viewFactor(orbit_altitude, radius_Planet):
    h = (orbit_altitude + radius_Planet)/radius_Planet
    return 1/(h**2)

def getQ_in(Q_sun_flux, Q_albedo_flux, Q_IR_flux, Q_int, absorptivity, Area_circle, orbit_altitude, radius_Planet):
    Q_sun = fluxToHeat(Q_sun_flux, absorptivity, Area_circle)
    # Albedo and IR have a view factor, F
    F = viewFactor(orbit_altitude, radius_Planet)
    Q_albedo = fluxToHeat(Q_albedo_flux*F, absorptivity, Area_circle)
    # ZI: here it should be IR absorptivity, i.e. emissivity used elsewhere
    Q_IR = fluxToHeat(Q_IR_flux*F, absorptivity, Area_circle)
    
    Q_tot_hot = Q_sun + Q_albedo + Q_IR + Q_int # total heat in for the hot case
    # print(Q_sun, Q_albedo, Q_IR, Q_int)
    Q_tot_cold = Q_IR # total heat in for cold case
    return Q_tot_hot, Q_tot_cold

def getEqTemps(absorptivity, emissivity, radius_sphere, Q_sun_flux, Q_albedo_flux, Q_IR_flux, Q_int, orbit_altitude, radius_Planet, sigma, KelToCel):
    Area_circle, Area_sphere = geometry(radius_sphere)

    Q_hot, Q_cold = getQ_in(Q_sun_flux, Q_albedo_flux, Q_IR_flux, Q_int, absorptivity, Area_circle, orbit_altitude, radius_Planet)
    print('Q hot and cold:', Q_hot, Q_cold)
    # calc temp if reaches equilibrium in sun
    Temp_hot = eqTemp(Q_hot, Area_sphere, emissivity, sigma)
    # calc temp if reaches equilibrium in dark side
    Temp_cold = eqTemp(Q_cold, Area_sphere, emissivity, sigma)
    return Temp_hot, Temp_cold, Q_hot, Q_cold

def geometry(radius_sphere):
    Area_circle = np.pi*(radius_sphere**2) # area of surface radiation sees
    Area_sphere = 4*np.pi*(radius_sphere**2) # area of sphere that emits heat
    return Area_circle, Area_sphere

def nextTemp(T, t_step, Qheating, c, mass, sigma, emissivity, Area_sphere):
    dTdt = (1/(c*mass))*(Qheating-(sigma*(T**4)*emissivity*Area_sphere))
    return T + dTdt*t_step

def getIR_Flux(stefanBoltzmann, emmissivity_object, temp_object):
    return stefanBoltzmann*emmissivity_object*(temp_object**4)

def TempPlot(time, temps, coating, modelT):
    # Plot the Temperature wrt time
    plt.figure()
    plt.plot(time,temps)
    plt.plot(modelT,temps, c='r')
    # print('timeRat:', time[-1]/modelT[-1])
    plt.title('Temperature on Dark Side for ' + coating)
    plt.xlabel('Time (s)')
    plt.ylabel('Temperature (K)')
    name = 'figures/TemperatureDarkSide_' + coating + '.png'
    plt.savefig(name)

def TempPlot2(time, temps, coating, modelT):
    # Plot the Temperature wrt time
    plt.figure()
    plt.plot(time,temps)
    plt.plot(modelT,temps, c='r')
    # print('timeRat:', time[-1]/modelT[-1])
    plt.title('Temperature on Sun Side for ' + coating)
    plt.xlabel('Time (s)')
    plt.ylabel('Temperature (K)')
    name = 'figures/TemperatureSunSide_' + coating + '.png'
    plt.savefig(name)


    
def TempsPlot(time, temp1, coating1, temp2, coating2, temp3, coating3):
    # Plot the Temperature wrt time
    battTmin = 5.0
    battTmax = 40.0
    tempmax = []
    tempmin = []
    for int in range(0,len(time)):
        tempmax.append(battTmax)
        tempmin.append(battTmin)
    timeMin = np.array(time)/60
    fig, ax = plt.subplots()
    plt.plot(timeMin,np.array(temp1)-KelToCel,label=coating1, lw=3)
    plt.plot(timeMin,np.array(temp2)-KelToCel,label=coating2, lw=3)
    plt.plot(timeMin,np.array(temp3)-KelToCel,label=coating3, lw=3)
    plt.plot(timeMin,tempmax, lw=1, c='black')
    plt.plot(timeMin,tempmin, lw=1, c='black')
    ax.fill_between(timeMin, tempmin, tempmax, alpha=0.1) 
    plt.legend()
    plt.title('Temperatures compared to battery operating temperature')
    plt.xlabel('Time (min)')
    plt.ylabel('Temperature (C)')
    name = 'figures/TempsVsOperatingTemp.png'
    plt.savefig(name)

def nonSteadyStateTemp(Tinitial,t_final_min,t_step,Qheating,c,mass,sigma,emissivity,Area_sphere):
    T = Tinitial # set the initial temp to the hot eq temp
    Temps = []
    Temps.append(T) # include initial temp in list
    time = []
    time.append(0) # python counts from zero
    # need smaller time step
    t_final = t_final_min*60 # convert minutes to seconds
    for i in range(1, int((t_final/t_step) + 1), 1): # sampling every second for 2 hours
        T = nextTemp(T, t_step, Qheating, c, mass, sigma, emissivity, Area_sphere)
        Temps.append(T)
        time.append(i*t_step)
    return time, Temps

def titlePrint(coating):
    print()
    print(coating + ' Sphere, Temperature Extremes')
    print('-------------------------------------------------------')


In [424]:
def doOneCase(coating,alpha,epsilon):
    titlePrint(coating)
    absorptivity = alpha
    emissivity = epsilon
    # print('Q:', Q_sun_flux, Q_albedo_flux, Q_IR_flux, Q_int)
    Temp_hot, Temp_cold, Q_hot, Q_cold = getEqTemps(absorptivity, emissivity, radius_sphere, Q_sun_flux, Q_albedo_flux, Q_IR_flux, Q_int, orbit_altitude, radius_Earth, sigma, KelToCel)
    printEqTemp('Hot', Temp_hot, KelToCel)
    printEqTemp('Cold', Temp_cold, KelToCel)
    # solve time variation
    Area_circle, Area_sphere = geometry(radius_sphere)
    #print('mass:', mass, 'Qcold:', Q_cold, 'Qhot:', Q_hot)
    
    Tstart = Temp_hot
    # need to enforce cyclic boundary condition with iterations
    # the duration of "sunshine", in units of eclipse time (=t_final_min)
    facSun = 2.0 
    for k in range(0,25):
        time, Temps = nonSteadyStateTemp(Tstart,t_final_min,t_step,Q_cold,c,mass,sigma,emissivity,Area_sphere)
        #print('Cold Extreme: ' + str('%.2f'%(Temps[-1])) + 'K, = ' + str('%.2f'%(Temps[-1] - KelToCel)) + u'\u2103')
        time2, Temps2 = nonSteadyStateTemp(Temps[-1],facSun*t_final_min,t_step,Q_hot,c,mass,sigma,emissivity,Area_sphere)
        #print('Hot Extreme: ' + str('%.2f'%(Temps2[-1])) + 'K, = ' + str('%.2f'%(Temps2[-1] - KelToCel)) + u'\u2103')
        #print('iter:', k, Tstart, Temps2[-1])
        Tstart = 0.5*(Tstart+Temps2[-1])
        #print('       new Tstart:', Tstart)
    
    # from diff. eq. solution     
    t0 = c*mass/sigma/Area_sphere/emissivity/Temp_cold**3
    modelTime = getTempSolution(np.array(Temps), t0, Temp_cold)
    TempPlot(time,Temps,coating,modelTime)
    
    # TempPlot2(time2,Temps2,coating)
    t0 = c*mass/sigma/Area_sphere/emissivity/Temp_hot**3
    modelTime2 = getTempSolution(np.array(Temps2), t0, Temp_hot)
    TempPlot2(time2,Temps2,coating,modelTime2)

    timeArr = np.array(time)
    time2shifted = np.array(time2)+60*t_final_min
    timeA = np.concatenate((time, time2shifted), axis=None)
    TempsA = np.concatenate((Temps, Temps2), axis=None)
    return timeA, TempsA

# given an array of temperatures, Tcold and t0, return model time grid
def getTempSolution(T, t0, Tcold):
    tau = T/Tcold
    t1 = 0.5*(np.arctan(tau)-np.arctan(tau[0]))
    t2 = 0.25*(np.log((tau+1)/(tau[0]+1)) - np.log((tau-1)/(tau[0]-1)))
    return t0*(t1+t2)

In [425]:
############## CONSTANTS #################
# constants
sigma = 5.6703*(10**-8) #stefan-boltzmann constant
# stefanBoltzmann = sigma
KelToCel = 273.15
radius_Earth = 6357 #[km]

# integration constants
t_step = 0.5 #length of time step in seconds
t_final_min = 30 #length of time in dark [min] (LEO)
 
# Moon (not needed)
Albedo_Ratio_Moon = 0.1
emmissivity_Moon = 0.97
temp_Moon = 384
radius_Moon = 1737 #[km]

# Sphere material properties
'''
---If you are looking to calculate the radius of a solid Al sphere,
rho = 2710 # Al density [kg/m^3]
mass = 100 [kg]
Volume = mass/rho
radius_sphere = (Volume*(3/4)*(1/np.pi))**(1/3)
'''
c = 921.1 # Al specific heat [J/(kg K)]
radius_sphere = 0.1 # 0.2 is close to solid 100kg Al sphere radius [m]
mass = 4/3*np.pi*radius_sphere**3 * 2710  # kg 
# mass is 11 kg for r=0.1m, but let's assume hollow sphere
mass = 4.0
#mass = 0.1

In [426]:
############# Run cases for different coatings ##############
# orbital parameters 
orbit_altitude = 550 #[km]
t_final_min = 30

# heat constants 
Q_sun_flux = 1367  # W/m2
Albedo_Ratio_Earth = 0.3
Q_albedo_flux = Q_sun_flux*Albedo_Ratio_Earth # solar flux reflected from Earth 
Q_IR_flux = 239  # W/m2,  IR flux emitted by Earth
Q_int = 10 # Watt, internal radiation from payload power usage  

timeA, TempsBlack = doOneCase('Al Black Anodized', 0.86, 0.86)
timeA, TempsBlue = doOneCase('Al Blue Anodized', 0.67, 0.87) 
timeA, TempsYellow = doOneCase('Al Yellow Anodized', 0.47, 0.87)  

TempsPlot(timeA, TempsBlack, 'Al Black Anodized', TempsBlue, 'Al Blue Anodized', TempsYellow, 'Al Yellow Anodized')

plt.close("all")


Al Black Anodized Sphere, Temperature Extremes
-------------------------------------------------------
Q hot and cold: 61.78862896525156 5.469803637740713
Hot Eq. Temp: 316.88K = 43.73℃
Cold Eq. Temp: 172.85K = -100.30℃

Al Blue Anodized Sphere, Temperature Extremes
-------------------------------------------------------
Q hot and cold: 50.34695512409134 4.261358648007301
Hot Eq. Temp: 300.20K = 27.05℃
Cold Eq. Temp: 161.92K = -111.23℃

Al Yellow Anodized Sphere, Temperature Extremes
-------------------------------------------------------
Q hot and cold: 38.30308792287004 2.989311290393181
Hot Eq. Temp: 280.37K = 7.22℃
Cold Eq. Temp: 148.19K = -124.96℃
